# Orbit Calibration Set Up  
This notebook will download the relevant data from space-track from the sp3 files that you provide. This will be then stored in the rawdata folder that can then be used for the calibration analysis. 

In [59]:
import os
import sp3
from datetime import datetime, timedelta
from dotenv import load_dotenv
import requests
import json
import spacetrack.operators as op
from spacetrack import SpaceTrackClient

st = SpaceTrackClient(identity='indigo.brownhall.20@ucl.ac.uk', password='Greeningplant123!')

In [7]:
# manually fill out the folder where your s3 data for grace-fo-1 and grace-fo-2 is stored
RawDataFolderName = 'v1'

# get the current working directory
cwd = os.getcwd()
RawDataFolder = os.path.join(cwd, 'RawData', RawDataFolderName)

# get all the files in the RawDataFolder
files = os.listdir(RawDataFolder)

# check the number with the *.sp3 extension
sp3files = [file for file in files if file.endswith('.sp3')]
if len(sp3files) != 2:
    print('Warning: There are {} files with the *.sp3 extension in the RawData folder. There should be 2.'.format(len(sp3files)))

# grace1 is the one that has L64 in the file name
grace1Name = [file for file in sp3files if 'L64' in file][0]
grace2Name = [file for file in sp3files if 'L65' in file][0]

In [9]:
# get the range of dates from the grace1 and grace2 files
grace1 = sp3.Product.from_file(os.path.join(RawDataFolder, grace1Name))
grace2 = sp3.Product.from_file(os.path.join(RawDataFolder, grace2Name))

In [12]:
# Get the smallest start date and the largest end date
start_date = min(grace1.satellites[0].records[0].time, grace2.satellites[0].records[0].time)
end_date = max(grace1.satellites[0].records[-1].time, grace2.satellites[0].records[-1].time)

# add 1 day buffer room
start_time_range = start_date - timedelta(days=1) # need some buffer room
end_time_range = end_date + timedelta(days=2)

print('Start date: {}'.format(start_time_range))
print('End date: {}'.format(end_time_range))

Start date: 2023-09-23 09:59:42+00:00
End date: 2023-09-27 00:00:12+00:00


In [78]:
# need to update the date to take out the time to be dynamic
grace1_tle = st.tle(norad_cat_id=43476, epoch='>2023-09-23 09:59:42&<2023-09-27 09:59:42',orderby='epoch asc', format='json')
grace2_tle = st.tle(norad_cat_id=43477, epoch='>2023-09-23 09:59:42&<2023-09-27 09:59:42',orderby='epoch asc', format='json')

print(grace1_tle)
# save in the RawData folder with a name of grace-fo-1-tle-43476.json
with open(os.path.join(RawDataFolder, 'grace-fo-1-tle-43476.json'), 'w') as f:
    f.write(grace1_tle)

with open(os.path.join(RawDataFolder, 'grace-fo-2-tle-43477.json'), 'w') as f:
    f.write(grace2_tle)

[{"COMMENT":"GENERATED VIA SPACETRACK.ORG API","ORIGINATOR":"18 SPCS","NORAD_CAT_ID":"43476","OBJECT_NAME":"GRACE-FO 1","OBJECT_TYPE":"PAYLOAD","CLASSIFICATION_TYPE":"U","INTLDES":"18047A","EPOCH":"2023-09-23 11:36:26","EPOCH_MICROSECONDS":"968608","MEAN_MOTION":"15.26850868","ECCENTRICITY":"0.0014899","INCLINATION":"88.9899","RA_OF_ASC_NODE":"335.8238","ARG_OF_PERICENTER":"30.2811","MEAN_ANOMALY":"329.9296","EPHEMERIS_TYPE":"0","ELEMENT_SET_NO":"999","REV_AT_EPOCH":"29699","BSTAR":"0.00015877","MEAN_MOTION_DOT":"0.00004227","MEAN_MOTION_DDOT":"0","FILE":"4014315","TLE_LINE0":"0 GRACE-FO 1","TLE_LINE1":"1 43476U 18047A   23266.48364547  .00004227  00000-0  15877-3 0  9990","TLE_LINE2":"2 43476  88.9899 335.8238 0014899  30.2811 329.9296 15.26850868296999","OBJECT_ID":"2018-047A","OBJECT_NUMBER":"43476","SEMIMAJOR_AXIS":"6863.370","PERIOD":"94.312","APOGEE":"495.461","PERIGEE":"475.010","DECAYED":"0"},{"COMMENT":"GENERATED VIA SPACETRACK.ORG API","ORIGINATOR":"18 SPCS","NORAD_CAT_ID":"4